# Data Analysis - (Social network and property calculator)
### Enrico Gavagnin

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import py_fort_myrmidon as fm
import datetime
import networkx as nx
import networkx.algorithms.community as nxc
import statistics
import scipy.stats as stats
import os
from datetime import datetime, date, timedelta
import pickle
import warnings
import json

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Set working directory and open list of myrmidon files
working_dir = '/media/ll16598/EG_DATA-7/NTM/'
myrm_list = sorted([ s for s in os.listdir(working_dir) if s[15:17] == '_z'])
myrm_list = [m for m in myrm_list if '_nn' in m]


## Test mode (optional)

To reduce the computational cost of the script, turn on the test mode option. This will run the analysis only on the first 2 myrmidon files. 

In [4]:
test_mode = False
#test_mode = True

## Home-range communities heatmap plotting

In [5]:
# Heatmap stack calculator

def compute_HM_stack(exp, start, end):
    
    # Number of ants
    N_ants = len(exp.Ants)    
    
    # Number of 2D bins
    bins_x = range(0,8000, 50)
    bins_y = range(0,6000, 50)
    
    # initialise HM-stack (bins_y * bins_x * N_ants)
    HM_stack = [np.zeros((len(bins_y)-1,len(bins_x)-1))]*N_ants

    # Populate HM stack (loop through ants)
    for i in fm.Query.ComputeAntTrajectories(exp,start=start, end=end,\
                                             maximumGap=fm.Duration(max_gap * 10**9)):
        HM_stack[i.Ant - 1] = HM_stack[i.Ant - 1] + np.histogram2d(i.Positions[:,2], 
                                                                   i.Positions[:,1], 
                                                                   bins=[bins_y, bins_x])[0]           
        
    return HM_stack

# Function which plot heatmaps of home-range
def PLOT_HM_partition(HM_stack, partition, fld_path, file_name):
    
    # Read number of communities
    n_partition = len(partition)
    
    # Read list of antIDs (needed for best partitions which don't use all the network)
    ant_list = list(frozenset().union(*list(partition)))
    
    # Initiate heatmap
    HM_part = {p: np.zeros(np.shape(HM_stack[0])) for p in range(n_partition)}
    
    # build background (by stacking the home-ranges of all the ants) 
    background = np.zeros((np.shape(HM_stack[0])[0], np.shape(HM_stack[0])[1], 3))
    for a in ant_list:
        for c in range(3):
            background[:, :, c] += HM_stack[a]
    for c in range(3):
        background[:, :, c][background[:, :, c]>0] = 1
        
        
    # Create figure with as many subplots as number of communitites
    fig, axs = plt.subplots(int(np.ceil(n_partition / 3)), 3, figsize=(15, 10))
    c = list(sns.color_palette()) + list(sns.color_palette("Set2"))

    # Loop through communitites
    for part_idx in range(n_partition):
        
        if n_partition >= 4:
            axs[part_idx // 3, part_idx % 3].imshow(background, interpolation='gaussian')
        else:
            axs[part_idx].imshow(background, interpolation='gaussian')

        # Stack the home-ranges of all the ants in partition part_idx
        part_size = 0
        for ant in ant_list:
            if ant in partition[part_idx]:
                part_size += 1
                HM_part[part_idx] += HM_stack[ant] 
   
        # Inititalise RGB map
        RGB = np.zeros((np.shape(HM_stack[0])[0], np.shape(HM_stack[0])[1], 4))
        
        # Visual adjustment
        mono_scale = HM_part[part_idx]
        mono_scale = mono_scale / np.nanpercentile(HM_part[part_idx], 98)
        mono_scale[mono_scale>1] = 1

        for rgb_idx in range(3): 
            RGB[:, :, rgb_idx] = c[part_idx][rgb_idx] 
        RGB[:, :, 3] = mono_scale

        # Plotting heatmap
        if n_partition >= 4:
            axs[part_idx // 3, part_idx % 3].imshow(RGB, interpolation='hamming') 
            axs[part_idx // 3, part_idx % 3].set_title('Community ' + str(part_idx+1) + ' - size: ' + str(part_size))
        else:
            axs[part_idx].imshow(RGB, interpolation='hamming') 
            axs[part_idx].set_title('Community ' + str(part_idx+1) + ' - size: ' + str(part_size))

    # remove axis 
    for idx_ax in range(int(np.ceil(n_partition / 3)) * 3):                
        if n_partition >= 4:
            axs[idx_ax // 3, idx_ax % 3].axis('off')
        else:
            axs[idx_ax].axis('off')
        
    plt.suptitle(file_name, fontweight="bold")
    plt.tight_layout()
    
    # Save figure
    plt.savefig(fld_path + file_name + '.png', facecolor='white', transparent=False)
    plt.close()

## Property calculator

In [6]:
def G_prop(G, exp, start, end, time_win, max_gap, name, var, nest_focus, PLOT_HM_check = False):
    
    # This function receive as input a social network and other parameters and return 
    # a dictionary with all the transmission properties computed
    # 
    # AS an option, it can call the function to plot the home-range heatmaps of the communtited detected
    
    
    # Compute connencted components
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)

    # Compute Largest Component
    GC = G.subgraph(Gcc[0])
    
    # ------------------------------------------------------------------------------ 
    # Best partition (Greedy Method)

    # Resolution parameter (see Greedy method documentation)
    res = 1
    
    if var != None:
        
        # Set fixed number of partition to 3
        fixed_p = 3
        
        # Read number of communities from file
        mode_p = mode_communities_dic[var['link_type']][name[11:14]][time_win]
    
    else: # trivial case for inititation of dataframe call
        fixed_p = 1
        mode_p = 1
    
    # Best partition (All network, unsupervised number of communities)
    best_partition = nxc.greedy_modularity_communities(G, weight='weight', resolution=res)
    
    # Best partition (Largest connected component only,  unsupervised number of communities)
    best_partition_CC = nxc.greedy_modularity_communities(GC, weight='weight', resolution=res)
   
    # Best partition (Largest connected component only,unspecified number of communities and 
    # with different resolution value)
    best_partition_CC_res_09 = nxc.greedy_modularity_communities(GC, weight='weight', resolution=0.9)
        
    # Best partition (Largest connected component with imposed number mode_p of communties)
    best_partition_CC_mode_p = nxc.greedy_modularity_communities(GC, weight='weight', cutoff=mode_p, best_n=mode_p, resolution=res)
    
    # Best partition (Largest connected component with imposed number 3 of communties)
    best_partition_CC_3p = nxc.greedy_modularity_communities(GC, weight='weight', cutoff=fixed_p, best_n=fixed_p, resolution=res)
        
    # ------------------------------------------------------------------------------ 
    
    # Plot heatmaps of home-ranges of communities (optional)
    if PLOT_HM_check:
        
        # Save HM_partition unsupervised
        directory = 'plots/HM_partition/unsup_mod/' + myrm_file[7:10]
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        attributes = '_' + var['link_type'] + '_' + str(time_win / 3600) + 'h_' + str(var['h'])
        PLOT_HM_partition(compute_HM_stack(exp, start, end), 
                          best_partition_CC, 
                          directory + '/', 
                          name[7:15] + attributes)
        
        # Save HM_partition 3 communities
        directory = 'plots/HM_partition/sup_mod3p/' + myrm_file[7:10]
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        attributes = '_' + var['link_type'] + '_' + str(time_win / 3600) + 'h_' + str(var['h'])
        PLOT_HM_partition(compute_HM_stack(exp, start, end), 
                          best_partition_CC_3p, 
                          directory + '/', 
                          name[7:15] + attributes)
        
        # Save HM_partition mode_p parition
        directory = 'plots/HM_partition/sup_modmp/' + myrm_file[7:10]
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        attributes = '_' + var['link_type'] + '_' + str(time_win / 3600) + 'h_' + str(var['h'])
        PLOT_HM_partition(compute_HM_stack(exp, start, end), 
                          best_partition_CC_mode_p, 
                          directory + '/', 
                          name[7:15] + attributes)
        
    # Return dictionary with all computed properties
    
    return {# replicate number
            'rep': int(name[8:10]),
            # experiment name
            'exp': name[11:15],
            # Start time
            'start': fm.Time.ToDateTime(start),
            # time aggregation window length (s)
            'time_win': time_win, 
            # focus on nest only (Boolean)
            'nest_focus': nest_focus,
            # maximum gap
            'max_gap': max_gap,
            # number of ants in the largest component
            'GC': GC.number_of_nodes(),
            # total number of ants
            'ants': G.number_of_nodes(),
            # size of communitites in the unsupervised community detection (largest connected component)
            'cMOD_communities': [len(best_partition_CC[i]) for i in range(len(best_partition_CC))],
            # size of communitites in the supervised community detection with mod_3 communtites (largest connected component)
            'cmpMOD_communities': [len(best_partition_CC_mode_p[i]) for i in range(len(best_partition_CC_mode_p))],
            # size of communitites in the supervised community detection with 3 communtites (largest connected component)
            'c3pMOD_communities': [len(best_partition_CC_3p[i]) for i in range(len(best_partition_CC_3p))],
            # size of communitites in the unsupervised community detection with different resolution (largest connected component)
            'cMODres09_communities': [len(best_partition_CC_res_09[i]) for i in range(len(best_partition_CC_res_09))],
            # Modularity coefficient for the unsupervised community detection (all network)
            'MOD': nxc.modularity(G, best_partition),
            # Modularity coefficient for the unsupervised community detection (largest connected component)
            'cMOD': nxc.modularity(GC, best_partition_CC),
            # Modularity coefficient for the supervised community detection with 3 communtites (largest connected component)
            'c3pMOD': nxc.modularity(GC, best_partition_CC_3p),
            # Modularity coefficient for the supervised community detection with mod_p communtites (largest connected component)
            'cmpMOD': nxc.modularity(GC, best_partition_CC_mode_p),
            # Density 
            'DEN': nx.density(G), 
            # Diameter (unweighted)
            'DIA': nx.diameter(GC),
            # Diameter (weighted)
            'wDIA': nx.diameter(GC, e=nx.eccentricity(GC, sp=dict(nx.shortest_path_length(GC,weight='inv_weight')))),
            # Radius (unweighted)
            'RAD': nx.radius(GC),
            # Radius (weighted)
            'wRAD': nx.radius(GC, e=nx.eccentricity(GC, sp=dict(nx.shortest_path_length(GC,weight='inv_weight')))),
            # Degree heterogeneity (All network)
            'DEH': np.std([G.degree(n) for n in G.nodes()]),
            # Degree heterogeneity (Largest connected component)
            'cDEH': np.std([GC.degree(n) for n in GC.nodes()]),
            # Weighted degree heterogeneity - ie Strength heterogeneity (All network)
            'wDEH': np.std(nx.adjacency_matrix(G).sum(axis=0)), 
            # Weighted degree heterogeneity - ie Strength heterogeneity (Largest connected component)
            'cwDEH': np.std(nx.adjacency_matrix(GC).sum(axis=0)),
            # Clustering (weighted)
            'CLS': np.mean([c for c in nx.clustering(G, weight='weight').values()]) 
            }

# initialise data-frame with properties
prop_df = pd.DataFrame(columns=G_prop(nx.star_graph(5),[],fm.Time.Now(),[],1,1,myrm_list[0],var=None, nest_focus=True).keys())

## Graph calculator

In [7]:
class Interaction:
    def __init__(self, duration, zone, start, end):
        self.duration = duration
        self.zone = zone
        self.start = start
        self.end = end

    def to_dict(self):
        return {
            'duration': self.duration,
            'zone': self.zone,
            'start': self.start,
            'end': self.end
        }

    def __repr__(self):
        return f"Duration: {self.duration}, Zone: {self.zone}, Start: {self.start}, End: {self.end}"

def compute_G(exp, start, end, min_cum_duration, frm_rate, link_type, nest_focus):
    
    # This function return the time-aggregated social network according to all 
    # the parameters received as input
    
    # Number of ants
    N_ants = len(exp.Ants)    

    ant_id1=[]
    ant_id2=[]
    durations=[]
    zones=[]
    time_start=[]
    time_end=[]
    interaction_list=[]
    interactions=fm.Query.ComputeAntInteractions(exp, start=start, end=end, maximumGap=fm.Duration(max_gap*10**9),
                                             reportFullTrajectories= False)[1]
    
    TimeToFrame = {fm.Time.ToTimestamp(frm[0].FrameTime): i + 1 
               for i, frm in enumerate(fm.Query.CollideFrames(exp,start=start,end=end))}
    for i in interactions:
        zone1=tuple(i.Trajectories[1].Zones)[0]
        try:
            zone2=tuple(i.Trajectories[1].Zones)[1]
            zone=zone2
        except IndexError:
            zone=zone1
        
        #zone 2 is the arena
        if nest_focus and zone1==2:
            continue
        if link_type == 'length_inter':
        # WEIGHTS: cumulative interaction time (s)
            weight=(TimeToFrame[fm.Time.ToTimestamp(i.End)] - \
                                                TimeToFrame[fm.Time.ToTimestamp(i.Start)]) * 1\
            / frm_rate
        elif link_type == '#inter':
            weight=1
        ant_id1.append(i.IDs[0])
        ant_id2.append(i.IDs[1])
        durations.append(weight)
        zones.append(zone)
        time_start.append(TimeToFrame[fm.Time.ToTimestamp(i.Start)] )
        time_end.append(TimeToFrame[fm.Time.ToTimestamp(i.End)] )
        interaction_list.append(tuple([i.IDs[0], i.IDs[1], weight, zone]))

    # transform into network
    G = nx.MultiGraph()

    # Add nodes and edges
    for i in range(len(ant_id1)):
        G.add_node(ant_id1[i])
        G.add_node(ant_id2[i])
        interaction = Interaction(durations[i], zones[i], time_start[i], time_end[i])
        G.add_edge(ant_id1[i], ant_id2[i], interaction=json.dumps(interaction.to_dict()))

    #multilayer component. First necessary to compute trajectories
    trajectories=fm.Query.ComputeAntTrajectories(exp, start=start,end=end, computeZones=True)
    for node in G.nodes():
        G.nodes[node]['zones'] = {}
    print('nodes in G:',len(G.nodes()))
    for t in range(0,len(trajectories)):
        tr=trajectories[t]
        ant_id=tr.Ant
        try:
            G.nodes[ant_id]
        except KeyError:
           # print(ant_id)
            continue
        pos=tr.Positions
        t_start=tr.Start
        duration=(tr.End()-tr.Start).Seconds()
        #print(ant_id)
        for i in range(0,len(pos)):
            p=pos[i]
            zone=p[4]
            if zone==2:
                continue
            start=p[0]
            e = pos[i+1][0] if i+1 < len(pos) else duration
            length=e-start
            if zone not in G.nodes[ant_id]['zones']:
                G.nodes[ant_id]['zones'][zone] = 0

            G.nodes[ant_id]['zones'][zone] += length
            #zone2=zone#here I need to get rate of switch and num ants in each zone at each time
            #I need t
    return G


## Main script

This script loops through all the myrmidon files. For each replicate it construct the social network, compute all the network properties and save them in a dataframe called prop_df.

In [8]:
save_G=True
save_df=False
compute_props=False

In [ ]:
## ========= PARAMETERS========= 
# Frame rate
frm_rate = 6

# max gap for different interactions (s)
max_gap = 10

# minimum interaction weight (OPTIONAL, set=0 if not wanted)
min_cum_duration = 0 

# Read file with modes of communities number computed   
if compute_props==True:
    mode_communities_dic = pd.read_pickle(r'data/mode_communities.pkl')

# ========= Edges weights type ==========
# Decide what type of weight for the edges of the social network 
#     'length_inter': cumulative interaction time (s))    
#     '#inter': number of interactions    

link_type = '#inter'

# ========= Nest focus zone ==========
# Set weather to only consider interactions happening within the nest zone
#     'True': Nest only
#     'False': All interactions considered
nest_focus = True

# Plot home-range heatmaps of communities (slow)
PLOT_HM = False

data_saving_name = 'data/polish_data_test' + link_type + '_NF_F.pkl'

# initialise data-frame with properties
prop_df = pd.DataFrame(columns=G_prop(nx.star_graph(5), [], fm.Time.Now(),
                                      [], 1, 1, myrm_list[0], var=None, 
                                      nest_focus=True).keys())

if test_mode:
    myrm_list = myrm_list[20:24]

# Loop through the myrmidon files
for myrm_file in myrm_list:

    # Skip replicate 41 (major escape), and all the replicates of the first 3 blocs (different tag orientation)
    if int(myrm_file[8:10])==41 or int(myrm_file[8:10])<13: 
        continue

    print(myrm_file)

    # Open experiment file
    exp = fm.Experiment.Open(working_dir + myrm_file)

    
    # Read the start date of the time window to be considered for the analysis
    # (ie the day before the experiment has been terminated)
    start_date = (fm.Time.ToDateTime(fm.Query.GetDataInformations(exp).End) +
                 timedelta(days = -1)).strftime("%Y-%m-%d")

    # ======== Total cumulative time window ========
    # Define the start/end of the entire time window to be considered to the analysis
    # ie from 9:00am of start_date to 9:00am of the following day
    
    start = fm.Time(datetime.fromisoformat(start_date + 'T09:00:00'))  
    end = start.Add(fm.Duration(60 * 60 * 24 * 10**9))
    
    # ======== Loop different time aggregated windows ========
    # Test different lengths of time aggregation (h)
    for time_win_h in [1]:
        
        print('Time_win:' + str (time_win_h) + 'h')
        
        # Loop through non-overlapping time windows of the given length
        for tw in range(0,25 - time_win_h, time_win_h):#range(2):#
            
            print('\rtime_slot: #' + str(tw + 1), end="")
            
            # Compute start/end of the time window
            s = start.Add(fm.Duration(tw * 60**2 * 10**9))
            e = start.Add(fm.Duration((tw + time_win_h) * 60**2 * 10**9))

            # Compute network
            G = compute_G(exp, s, e, min_cum_duration, frm_rate, link_type, nest_focus=nest_focus)

            # variables dict
            var = {'link_type': link_type, 'time_win': time_win_h * 60**2, 'h': tw}
            if compute_props==True:
                # Compute network properties and save them in prop_df
                prop_df = prop_df.append(G_prop(G, 
                                                exp, 
                                                s, e, 
                                                time_win_h * 60**2, 
                                                max_gap, 
                                                myrm_file, var = var, 
                                                nest_focus=nest_focus, 
                                                PLOT_HM_check = PLOT_HM), ignore_index=True)
            if save_G==True:
                for node in G.nodes():
                    if 'zones' in G.nodes[node]:
                        G.nodes[node]['zones'] = json.dumps(G.nodes[node]['zones'])
                nx.write_graphml(G, working_dir+'multilayer_networks_duration/'+myrm_file+ \
                 '_'+str(time_win_h)+'_'+str(tw + 1)+"_graph.xml")

if save_df==True:            
    # Save dataframe
    a_file = open(data_saving_name, "wb")
    pickle.dump(prop_df, a_file)
    a_file.close()

prop_df

EG_NTM_s13_DEHa_zones_luke_2023_nn.myrmidon
Time_win:1h
time_slot: #1

Computing ant trajectories:  98%|█████▉| 59/60 [00:02<00:00, 27.35tracked min/s]


nodes in G: 151
time_slot: #2

Computing ant trajectories:  98%|█████▉| 59/60 [00:02<00:00, 26.61tracked min/s]


nodes in G: 144
time_slot: #3

...

Computing ant trajectories:  98%|█████▉| 59/60 [00:02<00:00, 26.50tracked min/s]


nodes in G: 133
time_slot: #18

Computing ant trajectories:  98%|█████▉| 59/60 [00:02<00:00, 20.00tracked min/s]


nodes in G: 143
time_slot: #19

Colliding frames:  70%|███████████▏    | 42/60 [00:01<00:00, 42.50tracked min/s]

I now have multilayer networks where the zone attribute is the duration ant has spent in zone. Next to get the rate of switching and max density I need to get a df with the number of ants in a given zone at a time. I could get heatmap entropy also. And I could get number of interactions at a given time - although can be done with the temporal

In [ ]:
from collections import Counter
def compute_G(exp, start, end, min_cum_duration, frm_rate, link_type, nest_focus):
    
    # This function return the time-aggregated social network according to all 
    # the parameters received as input
    
    # Number of ants
    N_ants = len(exp.Ants)    

    ant_id1=[]
    ant_id2=[]
    durations=[]
    zones=[]
    time_start=[]
    time_end=[]
    interaction_list=[]
    interactions=fm.Query.ComputeAntInteractions(exp, start=start, end=end, maximumGap=fm.Duration(max_gap*10**9),
                                             reportFullTrajectories= False)[1]
    
    TimeToFrame = {fm.Time.ToTimestamp(frm[0].FrameTime): i + 1 
               for i, frm in enumerate(fm.Query.CollideFrames(exp,start=start,end=end))}
    
    for i in interactions:
        zone1=tuple(i.Trajectories[1].Zones)[0]
        try:
            zone2=tuple(i.Trajectories[1].Zones)[1]
            zone=zone2
        except IndexError:
            zone=zone1
        
        #zone 2 is the arena
        if nest_focus and zone1==2:
            continue
        ant_id1.append(i.IDs[0])
        ant_id2.append(i.IDs[1])
        durations.append(weight)
        zones.append(zone)
        time_start.append(TimeToFrame[fm.Time.ToTimestamp(i.Start)] )
        time_end.append(TimeToFrame[fm.Time.ToTimestamp(i.End)] )
    zone_interaction_counts = Counter(zones)
    
    zone_durations=[]
    zone_ids=[]
    zone_ant_ids=[]
    zone_starts=[]
    zone_ends=[]
    #multilayer component. First necessary to compute trajectories
    trajectories=fm.Query.ComputeAntTrajectories(exp, start=start,end=end, computeZones=True)
    for t in range(0,len(trajectories)):
        tr=trajectories[t]
        ant_id=tr.Ant
        pos=tr.Positions
        t_start=tr.Start
        duration=(tr.End()-tr.Start).Seconds()
        #print(ant_id)
        for i in range(0,len(pos)):
            p=pos[i]
            zone=p[4]
            if zone==2:
                continue
            start=p[0]
            e = pos[i+1][0] if i+1 < len(pos) else duration
            length=e-start
            zone_durations.append(length)
            zone_ids.append(zone)
            zone_ant_ids.append(ant_id)

    return G, zone_counts


## Computing mode of communitites
This script compute the mode of the number of communities abtained with unsupervised greedy algorithm and save the result in a file called mode_communities.pkl

In [8]:
# Mode of communities number with greedy algorithm 

# ========== FILTERING ============
reps_discarded = [20,41]
mode_communities_dic = {}

for data_file_name in ['data/prop_data_12_6_4_3_2_1_#inter_nest_21042022.pkl']:

    df = pd.read_pickle(data_file_name)
    
    if '_#inter_' in data_file_name:
        link_type = '#inter'
    elif '_length_inter_'in data_file_name:
        link_type = 'length_inter'
    
    mode_communities_dic[link_type] = {}

    for exp in ['MOD', 'DIA', 'DEN', 'DEH']:
        mode_communities_dic[link_type][exp] = {}
        
        for time_win in [1, 2, 3, 4, 6, 12]:
            
            df_filt = df.loc[(df.time_win==3600 * time_win) &  
                            (df.exp.isin([exp+'a',exp+'b'])) & 
                            (~df.rep.isin(reps_discarded))]
            
            mode_communities_dic[link_type][exp][time_win * 3600] = stats.mode([len(com) for com in df_filt.cMOD_communities]).mode[0]
            
a_file = open('data/mode_communities.pkl', "wb")
pickle.dump(mode_communities_dic, a_file)

## Connectivity Analysis

This script contains an auxiliary function which compute the size of the largest connected component as a funciton of the size time window of aggregation (Percolation analysis) 

NOT WELL DOCUMENTED

In [9]:
# Time vs Connectivity

def compute_Gcc(exp, start, end):
    
    # Number of ants
    N_ants = len(exp.Ants)    

    # initialise adj-matrix
    adj_mat = np.zeros((N_ants, N_ants))
    
    Gcc = [[sorted(nx.connected_components(nx.Graph(adj_mat)), key=len, reverse=True)],
           [0]]

    # Populate network
    for i in fm.Query.ComputeAntInteractions(exp,start=start,end=end,maximumGap=fm.Duration(max_gap*10**9),
                                            reportFullTrajectories= False)[1]:
        # Focus on Nest zone (id=1)
        if (1 in i.Trajectories[1].Zones) & (adj_mat[i.IDs[0]-1, i.IDs[1]-1] == 0):
            
            adj_mat[i.IDs[0]-1, i.IDs[1]-1] = 1           
         
            if i.Start.After(start.Add(fm.Duration((Gcc[1][-1] + 1) * 10**9))):
    
                Gcc[0].append(sorted(nx.connected_components(nx.Graph(adj_mat)), key=len, reverse=True))
                Gcc[1].append(Gcc[1][-1] + 1)
                    
    # network build
    return   Gcc

## ============= LOOP MYRMIDON =============== 

Gcc_df = pd.DataFrame(columns=['rep', 'exp', 'Gcc'])
max_gap = 10
for myrm_file in myrm_list:

    if int(myrm_file[8:10])==41 or int(myrm_file[8:10])<13: 
        continue

    print(myrm_file)

    # Open experiment file
    exp = fm.Experiment.Open(working_dir + myrm_file)

    ## ------ Time window ------ 
    start_date = (fm.Time.ToDateTime(fm.Query.GetDataInformations(exp).End) +
                 timedelta(days = -1)).strftime("%Y-%m-%d")

    start_time = fm.Time(datetime.fromisoformat(start_date + 'T09:00:00'))  
    Gcc = compute_Gcc(exp,start_time,start_time.Add(fm.Duration(12 * 60 * 60 * 10**9)))
    Gcc_df = Gcc_df.append({'rep': int(myrm_file[8:10]), 'exp': myrm_file[11:15], 'Gcc': Gcc}, ignore_index=True)
    
# Save
a_file = open("data/Gcc_NEST.pkl", "wb")
pickle.dump(Gcc_df, a_file)
a_file.close()


EG_NTM_s13_DEHa.myrmidon


Computing ant interactions: 100%|███▉| 719/720 [00:16<00:00, 43.40tracked min/s]


EG_NTM_s13_DEHb.myrmidon


Computing ant interactions: 100%|███▉| 719/720 [00:17<00:00, 40.48tracked min/s]


EG_NTM_s14_MODa.myrmidon


...

EG_NTM_s39_DENa.myrmidon


Computing ant interactions: 100%|███▉| 719/720 [00:20<00:00, 35.46tracked min/s]


EG_NTM_s39_DENb.myrmidon


Computing ant interactions: 100%|███▉| 719/720 [00:20<00:00, 35.66tracked min/s]


KeyboardInterrupt: 

In [ ]:

plot_fld_path = 'plots/Connectivity_analysis/'


for exp in ['DIA', 'MOD', 'DEH', 'DEN']:
    
    # Reading dataframe of connectivity
    Gcc_df = pd.read_pickle('data/Gcc_NEST.pkl')

    
    #Filter for the experiement of interest
    Gcc_df = Gcc_df[Gcc_df.rep != 20]
    Gcc_df = Gcc_df[[e[0:3] == exp for e in Gcc_df.exp]]
    Gcc_df.sort_values(['rep','exp'])
    connect_df = pd.DataFrame(columns=['rep', 'nest', 'time (min)', 'size_GCC', '#CC'])

    # treshold minimal partition size
    tresh_part = 2

    # Reformatting dataframe
    for row in range(len(Gcc_df)):
        df_aux = pd.DataFrame({'rep': [Gcc_df.rep.iloc[row]] * len(Gcc_df.Gcc.iloc[row][0]),
                              'nest': [Gcc_df.exp.iloc[row]] * len(Gcc_df.Gcc.iloc[row][0]),
                              'time (min)': Gcc_df.Gcc.iloc[row][1],
                              'size_GCC': [len(cc[0]) for cc in Gcc_df.Gcc.iloc[row][0]],
                              '#CC': [sum([len(p) > tresh_part for p in cc]) for cc in Gcc_df.Gcc.iloc[row][0]]})
        connect_df = pd.concat([connect_df, df_aux], ignore_index=True)

    # Plotting    
    sns.set(font_scale = 1.1)
    fig, axs = plt.subplots(1, 1, figsize=(15,10))
    title = 'Connectivity_exp_' + exp + ', reps: ' + str(int(len(Gcc_df)/2)) 
    supt = plt.suptitle(title, fontweight="bold")
    # Plot 1 (size GCC)
    sns.lineplot(data=connect_df, x="time (min)", y="size_GCC", hue="nest", ax=axs)
    axs.set_title('Size connected component')
    plt.tight_layout()
    axs_i_r = axs.inset_axes([0.6,0.1,0.35,0.4])

    # Plot 1 (number CC)
    sns.lineplot(data=connect_df, x="time (min)", y="#CC", hue="nest", ax=axs_i_r)
    axs_i_r.set_title('Number of connected components of size > ' + str(tresh_part))
    axs_i_r.set_xlim([1,40])
    axs_i_r.set_ylim(bottom=1)
    plt.tight_layout()

    # ====================================
    # Difference in size of GCC
    diff_connect_df = pd.DataFrame(columns=['rep', 'time (min)', 'diff_GCC'])

    def diff_list_GCC(A, B):
        A_len = [len(a) for a in A]
        B_len = [len(b) for b in B]
        return [A_len[i] - B_len[i] for i in range(min(len(A), len(B)))]

    # Reformatting dataframe
    for row in range(0, len(Gcc_df) -1, 2):
        diff_GCC = diff_list_GCC(Gcc_df.Gcc.iloc[row][0], Gcc_df.Gcc.iloc[row + 1][0])
        df_aux = pd.DataFrame({'rep': [Gcc_df.rep.iloc[row]] * len(diff_GCC),
                              'time (min)': range(len(diff_GCC)),
                              'diff_GCC': diff_GCC})
        diff_connect_df = pd.concat([diff_connect_df, df_aux], ignore_index=True)

    # Plotting    
    # Plot 3 (diff size GCC)
    axs_i_l = axs.inset_axes([0.2,0.1,0.35,0.4])
    sns.lineplot(data=diff_connect_df, x="time (min)", y="diff_GCC", ax=axs_i_l)
    axs_i_l.plot([0, 720], [0, 0], ':r')
    axs_i_l.set_title('Difference in size of connected component (' + Gcc_df.exp.iloc[0] + ' - ' + Gcc_df.exp.iloc[1] + ')')
    plt.tight_layout()

    file_name = 'Connectivity_exp_' + exp 

    #Savefig
    plt.savefig(plot_fld_path + file_name + '.png', facecolor='white', transparent=False)

